## Import

In [6]:
import pandas as pd
import numpy as np
import random
import time

import subprocess

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait

import warnings
warnings.filterwarnings('ignore')

## Open Internal Server

In [ ]:
# webui-user 바로가기 파일 열기
# 이미지 제너레이터 내부서버를 여는 코드
subprocess.call(["start", "webui-user.Ink"])

In [ ]:
# Wait for the message "Model loaded" to appear in the batch file output
while True:
    try:
        with open("batch_file_output.txt") as output_file:
            output = output_file.read()
            if "Model loaded." in output:
                break
    except FileNotFoundError:
        pass
    
    # Wait for a short period of time before checking again
    time.sleep(1)

## Fine-tuning tags

In [18]:
taglist = pd.read_csv('TagList.csv')
taglist = taglist.drop(columns=taglist.columns[0], axis=1)
taglist

,"('기본', '작화')","('기본', '테마')","('기본', '초점')","('기본', '연출')","('기본', '배경')","('기본', '장소')","('작품', '작품')","('작품', '캐릭터')","('인물', '인원')","('인물', '인물')",...,"('동작 2', '브래지어')","('동작 2', '팬티')","('동작 2', '양말')","('동작 2', '모자')","('동작 2', '안경')","('동작 2', '마스크')","('동작 2', '넥타이')","('동작 2', '귀걸이')","('행동', '동사')","('행동', '동명사')"
0,manga_photorealistic,City_Pop,feet_out_of_frame,multiple_4koma,streaked,megastructure,Sword_Art_Online,White_Fox,6others,multiple_births,...,bra_on_ass,panties_on_head,taut_legwear,backwards_hat,eyewear_view,holding_mask,necktie_grab,adjusting_earrings,waving,shaking
1,Graphic_Novel,graphite,thigh_focus,1koma,striped,lake,Danganronpa,Artoria_Pendragon,3girls,Younger_brother,...,bra_removed,holding_panties,mismatched_legwear,sideways_hat,eyewear_hang,mask_on_head,NaN,NaN,splashing,thinking
2,Cartoon_realistic,Funk,looking_down,2koma,sky_blue,locker_room,MAPPA_Corporation,Orsola_Aquinas,1boy,father_and_son,...,bra_pull,adjusting_panties,torn_legwear,hat_over_one_eye,eyewear_on_head,removing_mask,NaN,NaN,slipping,scratching
3,Cartoon_photorealistic,Pop,ass_focus,borderless_panels,print,convention,Fairy_Tail,Itsuwa,2girls,Younger_Sister,...,bra_lift,panties_aside,wet_legwear,adjusting_headwear,looking_over_eyewear,mask_removed,NaN,NaN,bumping,slashing
4,Monochrome,Adult_Video,hip_focus,3koma,blue,volcano,Pokemon,Artoria_Caster,1girl,aunt_and_nephew,...,holding_bra,hand_in_panties,removing_legwear,hat_over_eyes,eyewear_on_headwear,mask_over_one_eye,NaN,NaN,elbowing,imagining
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
584,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Calamity_Jane,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sword_Maiden,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jaguar_Man,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mito_Uzumaki,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


기본적으로 들어가는 태그들은 https://novelai.io/tags 에서 크롤링해서 사용함. 다른 태그를 넣고싶다면 TagList.csv를 수정해서 넣을 수 있음.  

19금 태그는 일반적인 이미지 제너레이터에서 사용할 수 없기도 하고, 좋은 퀄리티의 그림을 생성하기 어려워서 19금 태그는 제외함.  
*TODO: 근데 다 제거하진 못해서 선정적인 단어 분류기같은 걸로 한 번 거르거나, 수동으로 걸러줘야할듯?*

('기본', '품질') 태그는 masterpiece, best quality ONLY로 고정해서 제외.

## Tag generation function

In [19]:
def get_tag(tagLists=taglist) -> str:
    '''
    태그 모음 DataFrame에서 랜덤하게 태그를 뽑아 태그 조합을 생성하는 함수입니다. 
    DataFrame의 Column들을 무작위로 조합해서 태그 문장을 만듭니다. 
    masterpiece, best quality는 무조건 포함됩니다.
    '''

    result = 'masterpiece, best quality, '
    columns = random.sample(list(tagLists.columns), k=random.randint(8, 12))
    
    for column in columns:
        column_values = tagLists[column].dropna().tolist()
        tag = random.choice(column_values)
        result += tag + ', '
        
    return result[:-2]

In [23]:
# TEST
get_tag()

'masterpiece, best quality, panda_ears, chasing, slender_hand, baseball_glove, floating, small_nose, black_man, eyewear_switch, Porno_Actor, inward_v'

## Creating images

In [4]:
URL = 'http://127.0.0.1:7860'
driver = webdriver.Edge(executable_path='msedgedriver.exe')
driver.get(url=URL)

# Novel AI 이외의 모델을 사용하려면
# 모델을 선택한 뒤 다음 셀 실행

C:\Users\36dav\AppData\Local\Temp\ipykernel_24780\2315313369.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge(executable_path='msedgedriver.exe')


WebDriverException: Message: unknown error: net::ERR_CONNECTION_REFUSED
  (Session info: MicrosoftEdge=109.0.1518.70)
Stacktrace:
Backtrace:
	Microsoft::Applications::Events::EventProperties::SetProperty [0x00007FF7B8F71672+15186]
	Microsoft::Applications::Events::EventProperty::EventProperty [0x00007FF7B8F0A1C2+827458]
	(No symbol) [0x00007FF7B8BCED90]
	(No symbol) [0x00007FF7B8BC6D72]
	(No symbol) [0x00007FF7B8BBBA41]
	(No symbol) [0x00007FF7B8BBCFE2]
	(No symbol) [0x00007FF7B8BBBC74]
	(No symbol) [0x00007FF7B8BBADB9]
	(No symbol) [0x00007FF7B8BBACF4]
	(No symbol) [0x00007FF7B8BB95B3]
	(No symbol) [0x00007FF7B8BB9D1A]
	(No symbol) [0x00007FF7B8BD0EA2]
	(No symbol) [0x00007FF7B8C4C041]
	(No symbol) [0x00007FF7B8C31F4A]
	(No symbol) [0x00007FF7B8C04C3E]
	(No symbol) [0x00007FF7B8C4B513]
	(No symbol) [0x00007FF7B8C31D23]
	(No symbol) [0x00007FF7B8C03B80]
	(No symbol) [0x00007FF7B8C02B0E]
	(No symbol) [0x00007FF7B8C04344]
	Microsoft::Applications::Events::EventProperties::SetProperty [0x

In [ ]:
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#my-element'))
)

buttonGen = driver.find_element(By.CSS_SELECTOR,
    'button#txt2img_generate.gr-button.gr-button-lg.gr-button-primary')


In [ ]:
driver.close()